# Templates

Base template defines the overall site behavior. It includes a navigation bar that links to each of the three pages on the site.
home has a single button that will clear the database.
form is the template for querying. It has code to display messages, which I use to display the results of a query.
file_up is the template for the file upload page. It describes the form used to upload files.

In [1]:
%%writefile templates/base.html
<!DOCTYPE html>
<html>
    <head>
        <title>Super Awesome Bibliography Query Site</title>
    </head>

    <body>
        <h1>{{ page_title }}</h1>
        
        <h2>
    <a href="{{ url_for('home') }}">Home</a>
    <a href="{{ url_for('query_db') }}">Query Database</a>
    <a href="{{ url_for('upload_file') }}">Upload a File</a>
    </h2>

    {{ message | safe}}        
    {% block content %}{% endblock %}        
    </body>
</html>


Overwriting templates/base.html


In [2]:
%%writefile templates/home.html
{% extends "base.html" %}
{% block content %}
<form action="/home/" method="POST">
            <input type="submit" name = 'Clear_Database' value = 'Clear Database'/>
</form>
{% endblock %}

Overwriting templates/home.html


In [3]:
%%writefile templates/form.html
{% extends "base.html" %}
{% block content %}
<p>
            Query the database using sql query syntax, entering the string that follows the WHERE.
            </br>
            Column names are 'ReferenceTag','Collection', 'Title', 'Author','Journal', 'Keywords',
              'Pages', 'Volume', 'Year' and 'id'.</br>
            To use wildcards, use LIKE and % as a multiple character wildcard and _ as a single
            character wildcard. </br>
            Explicitly put strings in quotes. Note that the pages column is a string.</p>
            <form action="/query_database/" method="POST">
            Make a query: 
            <input type="text" name="query" />
            <input type="submit" value ="Submit Query"/>
            </form>
            {% with messages = get_flashed_messages() %}
            {% if messages %}
    <ul class=flashes>
    {% for message in messages %}
      <li>{{ message | safe}}</li>
    {% endfor %}
    </ul>
  {% endif %}
{% endwith %}
{% endblock %}

Overwriting templates/form.html


In [4]:
%%writefile templates/file_up.html
{% extends "base.html" %}

{% set page_title = 'Choose file to upload' %}
{% block content %}
   
      <form action = "/upload_file/" method = "POST" 
         enctype = "multipart/form-data">
            Collection Name:
        <input type = "text" name = "collection_name" />
        File:
         <input type = "file" name = "file" />
         <input type = "submit" value = "Upload File"/>
      </form>
      
{% endblock %}

Overwriting templates/file_up.html


# Config File

In [5]:
%%writefile config.py

import os
basedir = os.path.abspath(os.path.dirname(__file__))

class Config(object):
    # ...
    SQLALCHEMY_DATABASE_URI = \
        'sqlite:///' + os.path.join(basedir, 'app.db')
    SQLALCHEMY_TRACK_MODIFICATIONS = False
    SECRET_KEY = 'Super Secret PASSWORD'
    DEBUG = True

Overwriting config.py


# General Application File
Initializes app, imports db from database file, and connects it to the app. Has three methods for each of the three site pages.

In [8]:
%%writefile basic_site.py

from flask import Flask, render_template, request, url_for, redirect, flash
import flask
import pybtex.database as pb
from flask_sqlalchemy import SQLAlchemy
from config import Config
import numpy as np
from database import db, Citation
from sqlalchemy.exc import OperationalError

app = Flask(__name__)
app.config.from_object(Config)
with app.app_context():
    db.init_app(app)
    db.create_all()

@app.route("/")
@app.route("/home")
@app.route("/home/", methods=['GET', 'POST'])
def home():
    """Welcomes the user and tells them how many collections are in the database
    If post, clears the database."""
    if request.method == 'POST':  #clear the database if the button was pressed
        with app.app_context():
            db.drop_all()
            db.create_all()
        this_message = 'Database cleared. Upload a new file to get started!'
        
    elif Citation.query.first() is None:  #check for empty db
        this_message = 'You have no collections. Upload a file to get started!'
    else:  #get number of collections and names
        collections = \
            np.unique(Citation.query.with_entities(Citation.Collection).all())
        colls_w_sizes = \
            [s+' (%s entries)' %get_size_collection(s) for s in collections]
        this_message = ('You have %s collections, with names : ' % len(collections)) + \
                        ', '.join([s for s in colls_w_sizes]) 
            
    return render_template('home.html', page_title="Home",
                           message="Welcome! </p>This is a site for querying" +
                               "bibtex files </p>"+ this_message)


@app.route('/query_database/', methods=['GET', 'POST'])
def query_db():
    """Displays the form.html page with a form for querying. If POST, attempts
    to query the database and if the query provides results, displays them using
    message flashing."""
    if request.method == 'POST':
        query = request.form['query']
        #given a query, add the sql prefix stuff to it
        orig_query = query
        query = 'SELECT * FROM bibliography WHERE ' + query
        if query not in (""," "):
            try:
                result=db.engine.execute(query).fetchall()
            except OperationalError:
                return render_template('form.html', page_title= 'Query Database', 
                                   message='Invalid query. Please try again.')
            if len(result) > 0:
                display_query_results(result)
            else:
                flash('No results to display :(')
            return render_template('form.html', page_title= 'Results for : ' + orig_query)
        else:
            return render_template('form.html', page_title= 'Query Database', 
                                   message='Invalid query. Please try again')
    else:  #if GET, display page
        return render_template("form.html", page_title = 'Query Database')
    
@app.route('/upload_file/', methods=['GET', 'POST'])
def upload_file():
    """Has a form to upload a file and provide a collection name. Tests
    that collection name + file were provided, then uses helper methods
    to add the bibtex file to the database"""
    if request.method == 'POST':
        if 'file' not in request.files:
            return render_template('file_up.html', message = 'Please Choose a File')
        file = request.files['file']
        coll_name = request.form['collection_name']
        if coll_name not in (""," "):
            add_file_to_db(coll_name, file)
            return render_template('base.html', page_title='Added collection: ' + coll_name)
        else:
            return render_template('file_up.html',
                                   message = 'Please enter a valid collection name')
    else:
        ## this is a normal GET request
        return render_template("file_up.html", page_title = 'Upload a File :)')

def display_query_results(results):
    """flashes a message for each of the fields to be displayed. These are then
    displayed by the template as bullets"""
    fields = ['ReferenceTag','Collection', 'Title', 'Author','Journal', 'Keywords',
              'Pages', 'Volume', 'Year']
    for row in results:
        for field in fields:
            if field == 'Year':  #for the final row, and a break
                flash(field + ': ' + str(row[field]) + '</br></br>')
            else:
                flash(field + ': ' + str(row[field]))
                
def add_file_to_db(coll_name, file):
    """uses pybtex to parse the file, then iterates through entries and adds them
    to the database"""
    parsed_file=pb.parse_bytes(file.read(), bib_format='bibtex')
    for k, entry in parsed_file.entries.items():
        add_entry_to_db(k, entry, coll_name)
                
def add_entry_to_db(key, entry, collection):
    """creates a keywords dictionary that is used to pass arguments to Citation,
    so that I don't have to pass each field individually. Deals with the oddities
    of pybtex's author storage and creates a list of authors"""
    keywords = {}
    for field in ['Title', 'Journal', 'Keywords', 'Pages', 'Volume', 'Year']:
        if field in entry.fields:
            keywords[field] = format_btex_fields(entry.fields[field])
        else:
            keywords[field] = None
    #handle author separately because its stored differently
    if 'Author' in entry.persons:
        author_list = [format_btex_fields(str(auth)) for auth in entry.persons['Author']]
        keywords['Author'] = ', '.join(author_list)
    else:
        keywords['Author'] = None
    with app.app_context():
        db.session.add(Citation(**keywords, Collection = collection, ReferenceTag = key))
        db.session.commit()
    
def format_btex_fields(string):
    """helper method to remove odd formatting from btex files"""
    string = string.replace('{','')
    string = string.replace('}','')
    return string

def get_size_collection(collection):
    """helper method to get the number of entries in a collection"""
    return len(Citation.query.filter(Citation.Collection == collection).all())

if __name__ == "__main__":
    app.run()

Overwriting basic_site.py


# Database Organization File

Initializes the database and the model it uses.

In [9]:
%%writefile database.py

from flask_sqlalchemy import SQLAlchemy

db = SQLAlchemy()

class Citation(db.Model):
    __tablename__ = 'bibliography'
    id = db.Column(db.Integer, primary_key=True)
    ReferenceTag = db.Column(db.String)
    Collection = db.Column(db.String)
    Author = db.Column(db.String)
    Journal = db.Column(db.String)
    Keywords = db.Column(db.String)
    Pages = db.Column(db.String)
    Title = db.Column(db.String)
    Volume = db.Column(db.Integer)
    Year = db.Column(db.Integer)


Overwriting database.py


In [13]:
%run basic_site.py

SyntaxError: invalid syntax (basic_site.py, line 36)

# Testing Code

In [13]:
%%writefile testing_test.py
import os
import unittest
from basic_site import app, db, Citation, add_file_to_db
from io import BytesIO
from flask_sqlalchemy import SQLAlchemy

class FlaskTestCase(unittest.TestCase):

    def setUp(self):
        self.app = app.test_client()
        with app.app_context():
            db.drop_all()
            db.create_all()
    
    def test_home_page(self):
        """Checks that homepage displays welcome text"""
        rv = self.app.get('/')
        assert b'Welcome!' in rv.data
    
    def submit_query(self, query):
        """helper function to test querying"""
        return self.app.post('/query_database/', data=dict(
            query=query
        ), follow_redirects=True)
    
    def test_query_page(self):
        """tests that the get method of query database contains the form"""
        rv = self.app.get('/query_database/')
        assert b'name="query"' in rv.data
    
    def test_upload_file_page(self):
        """tests that the 'get' page for upload file is displayed correctly"""
        rv = self.app.get('/upload_file/')
        
        assert b'Choose file to upload' in rv.data
        assert b'Upload a File' in rv.data
    
    def upload_file(self, coll_name, file, filename):
        """helper method to mimic the user choosing a file and collection name"""
        data = {
            'collection_name': coll_name,
            'file': (file, filename)
        }

        return self.app.post('/upload_file/', buffered=True,
                         content_type='multipart/form-data',
                         data=data)
        
    def test_file_upload(self):
        """Test that file upload displays the correct page and actually adds to database"""
        rv = self.upload_file('ex', open('hw_8/hw_8_data/homework_8_refs.bib','rb'),
                              'hw_8/hw_8_data/homework_8_refs.bib')
        assert rv.status_code == 200
        assert b'ex' in rv.data
        with app.app_context():
            assert len(Citation.query.all()) > 0
    
    def test_query_submission(self):
        """tests invalid queries and queries to empty databases"""
        assert b':(' in self.submit_query('Year < 1930').data
        assert b'Invalid' in self.submit_query("").data
        assert b'Invalid' in self.submit_query(" ").data
        assert b'Invalid' in self.submit_query("sentisoints").data
    
    def test_bad_file_upload(self):
        """tests failure to provide collection names or to choose a file"""
        rv = self.upload_file('',open('hw_8/hw_8_data/homework_8_refs.bib','rb'),
                              'hw_8/hw_8_data/homework_8_refs.bib')
        assert b'enter a valid collection name' in rv.data        
        data = {'collection_name': 'ex' }

        rv = self.app.post('/upload_file/', buffered=True,
                         content_type='multipart/form-data',
                         data=data)
        assert b'Please Choose a File' in rv.data

    def test_homepage_text(self):
        """test that the homepage properly displays collection info"""
        rv = self.app.get('/home/')
        assert b'no collections' in rv.data
    
    def test_homepage_text_not_empty(self):
        """add collections and test that it shows them and entries"""
        add_file_to_db('ex', open('hw_8/hw_8_data/homework_8_refs.bib','rb'))
        add_file_to_db('ex2', open('hw_8/hw_8_data/homework_8_refs.bib','rb'))
        rv = self.app.get('/home/')
        assert b'ex (46 entries)' in rv.data
        assert b'ex2 (46 entries)' in rv.data
        
    def test_real_query(self):
        """After adding a collection, check that the query produces the 
        expected results."""
        add_file_to_db('ex', open('hw_8/hw_8_data/homework_8_refs.bib','rb'))
        rv = self.submit_query('Author LIKE "%Dean%"')
        assert b'Dean' in rv.data
        assert b'Reddenings of Cepheids' in rv.data
    
    def test_db_clearing(self):
        """test that database clearing works as expected by adding file to database then clearing it"""
        rv = self.upload_file('ex', open('hw_8/hw_8_data/homework_8_refs.bib','rb'),
                              'hw_8/hw_8_data/homework_8_refs.bib')
        with app.app_context():
            assert len(Citation.query.all()) > 0
        #check that a random query doesn't clear the database
        self.app.get('/query_database/')
        with app.app_context():
            assert len(Citation.query.all()) > 0
        #then try to clear the database
        self.app.post('/home/')
        with app.app_context():
            assert len(Citation.query.all()) == 0

if __name__ == '__main__':
    unittest.main()

Overwriting testing_test.py
